In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/DS Grad School/DS8009 Graph Mining/Assign5/')

In [3]:
ls

Assignment5_Leo.ipynb  CA-AstroPh.txt


In [0]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools as IT


In [0]:
df = pd.read_csv('CA-AstroPh.txt', delimiter='\t', header=None)

In [0]:
G = [tuple(x) for x in df.to_records(index=False)]

In [12]:
#set up edges into dictionary
graph_dict = {}

for a,b in G:
  if a in graph_dict:
    graph_dict[a].append((b))
  else:
    graph_dict[a] = [(b)]

for a,b in G:
  if b in graph_dict:
    graph_dict[b].append((a))
  else:
    graph_dict[b] = [(b)]

len(graph_dict)

18772

#1. Implement from scratch any 3 proximity metrics (choosing at least 1 from each of local and global metrics) (7)

In [0]:
#Local - common neighbours
def common_neighbour(graph,x,y):

  x_nodes = graph[x]
  y_nodes = graph[y]

  return(len(list(set(x_nodes).intersection(y_nodes))))

In [14]:
common_neighbour(graph_dict,84424,33040)

9

In [0]:
#Local - preferential Attachment Index

def pref_attachIndex(graph,x,y):
  x_len = len(graph_dict[x])
  y_len = len(graph_dict[y])
  return(x_len*y_len)

In [16]:
pref_attachIndex(graph_dict,84424,33040)

77100

In [0]:
#Local - Jaccard Coefficient

def Jaccard_coef(graph,x,y):
  intersect = len(set(graph_dict[x]).intersection(graph_dict[y]))
  union = len(set(graph_dict[84424]))+len(set(graph_dict[33040]))
  return( intersect/union )


In [18]:
Jaccard_coef(graph_dict,84424,33040)

0.02710843373493976

#2. Which metric performs the best for the given graph (compare performance on accuracy and f1-score). (You will need to implement the link prediction and measurement framework from scratch (8)

In [0]:
#read data as a graph
Graph = nx.Graph()

for edges in G:
  Graph.add_edge(*edges)

In [0]:
#identify and finds all missing edges this will be my label 0
missing = [pair for pair in IT.combinations(Graph.nodes(), 2)
           if not Graph.has_edge(*pair)]

In [33]:
#set up my missing edges as label 0, i used the first 100K
df_train_mis = pd.DataFrame(missing[0:100000])

#calculate all the attributes for test set
comm_neigh3 = []
pref_index3 = []
jaccard3 = []
label3 = []
for index, row in df_train_mis.iterrows():
  comm_neigh3.append(common_neighbour(graph_dict,row[0],row[1]))  
  pref_index3.append(pref_attachIndex(graph_dict,row[0],row[1]))
  jaccard3.append(Jaccard_coef(graph_dict,row[0],row[1]))
  label3.append(0)

df_train_mis['common_neigh'] = comm_neigh3
df_train_mis['pref_index'] = pref_index3
df_train_mis['jaccard'] = jaccard3
df_train_mis['label'] = label3

df_train_mis

,0,1,common_neigh,pref_index,jaccard,label
0,84424,560,1,1800,0.003012,0
1,84424,15829,1,4500,0.003012,0
2,84424,42972,3,21600,0.009036,0
3,84424,55528,2,900,0.006024,0
4,84424,57618,2,9900,0.006024,0
...,...,...,...,...,...,...
99995,39193,61795,0,528,0.000000,0
99996,6229,77504,0,5544,0.000000,0
99997,6229,77973,0,7728,0.000000,0
99998,6229,85107,0,3864,0.000000,0


In [0]:
#shuffle my data
df_shuffle = df.sample(frac=1)

In [0]:
#set my split to 0.8 : 0.2
msk = np.random.rand(len(df_shuffle)) < 0.8
train = df_shuffle[msk]
test = df_shuffle[~msk]

In [37]:
test

,0,1
395404,12757,28002
1484,101811,74444
349781,69191,101890
372327,103785,111622
326074,113663,4422
...,...,...
202093,72201,35290
254017,61750,37519
181966,38921,111385
365511,32670,21005


In [41]:
#setting up my train dataset with label 1
comm_neigh = []
pref_index = []
jaccard = []
label = []

for index, row in train.iterrows():
  comm_neigh.append(common_neighbour(graph_dict,row[0],row[1]))
  pref_index.append(pref_attachIndex(graph_dict,row[0],row[1]))
  jaccard.append(Jaccard_coef(graph_dict,row[0],row[1]))
  label.append(1)

train['common_neigh'] = comm_neigh
train['pref_index'] = pref_index
train['jaccard'] = jaccard
train['label'] = label

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [42]:
#combine my missing edges with my train dataset that i split for me to train
df_train2 = pd.concat([train,df_train_mis])
df_train2

,0,1,common_neigh,pref_index,jaccard,label
390044,18357,102756,10,484,0.030120,1
184810,116724,66401,31,6144,0.093373,1
314189,51934,44329,14,1140,0.042169,1
338439,21086,4815,2,636,0.006024,1
40380,57618,105016,7,7392,0.021084,1
...,...,...,...,...,...,...
99995,39193,61795,0,528,0.000000,0
99996,6229,77504,0,5544,0.000000,0
99997,6229,77973,0,7728,0.000000,0
99998,6229,85107,0,3864,0.000000,0


In [43]:
#calculate all the attributes for test set
comm_neigh2 = []
pref_index2 = []
jaccard2 = []

for index, row in test.iterrows():
  comm_neigh2.append(common_neighbour(graph_dict,row[0],row[1]))  
  pref_index2.append(pref_attachIndex(graph_dict,row[0],row[1]))
  jaccard2.append(Jaccard_coef(graph_dict,row[0],row[1]))

test['common_neigh'] = comm_neigh2
test['pref_index'] = pref_index2
test['jaccard'] = jaccard2

test


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,0,1,common_neigh,pref_index,jaccard
395404,12757,28002,4,1540,0.012048
1484,101811,74444,2,6384,0.006024
349781,69191,101890,2,444,0.006024
372327,103785,111622,5,192,0.015060
326074,113663,4422,4,5280,0.012048
...,...,...,...,...,...
202093,72201,35290,32,68320,0.096386
254017,61750,37519,1,832,0.003012
181966,38921,111385,25,75828,0.075301
365511,32670,21005,1,88,0.003012


In [0]:
columns = ['common_neigh','pref_index', 'jaccard']
x = df_train2[columns]
y = df_train2['label']

In [45]:
#train my data
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=30, max_depth=5, random_state=0)
RF.fit(x,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
columns = ['common_neigh','pref_index', 'jaccard']
predictions = RF.predict(test[columns])

In [47]:
test['label'] = predictions

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
test

,0,1,common_neigh,pref_index,jaccard,label
395404,12757,28002,4,1540,0.012048,1
1484,101811,74444,2,6384,0.006024,1
349781,69191,101890,2,444,0.006024,1
372327,103785,111622,5,192,0.015060,1
326074,113663,4422,4,5280,0.012048,1
...,...,...,...,...,...,...
202093,72201,35290,32,68320,0.096386,1
254017,61750,37519,1,832,0.003012,1
181966,38921,111385,25,75828,0.075301,1
365511,32670,21005,1,88,0.003012,1


In [52]:
#lets see how many we got correct from all the connected nodes from our test set
sum(predictions)

77495

In [55]:
#in our case we tested on only cases of connected nodes therefore there are no false Negative or False positive 

Precision = 77495/ 78660
Recall = 77495 / 78660
Accuracy = 77495 / 78660
F1_Score = 2* (Precision*Recall) / (Precision+Recall)

Precision, Recall, Accuracy, F1_Score


(0.9851894228324434,
 0.9851894228324434,
 0.9851894228324434,
 0.9851894228324434)